# Import packages and set parameters

In [ ]:
import numpy as np
from numpy import linalg
from numba import jit
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.transforms as mtransforms
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)

from scipy.constants import pi
from scipy.interpolate import RectBivariateSpline, griddata

import os
import ast
import pandas as pd

import mpl_scatter_density # adds projection='scatter_density'
from matplotlib.colors import LinearSegmentedColormap

def set_pandas_display_options() -> None:
    """Set pandas display options."""
    # Ref: https://stackoverflow.com/a/52432757/
    display = pd.options.display

    display.max_columns = None
    display.max_rows = None
    display.max_colwidth = None
    display.width = None
    display.chop_threshold = None
    display.precision = 14  # set as needed

set_pandas_display_options()

In [ ]:
palette = list(sns.color_palette("Blues", 6).as_hex())

# "Inferno-like" colormap with white background
white_blues = LinearSegmentedColormap.from_list('white_blues', [
    (0, '#ffffff'),
    (1e-10, palette[0]),
    (0.2, palette[1]),
    (0.4, palette[2]),
    (0.6, palette[3]),
    (0.8, palette[4]),
    (1, palette[5]),
], N=256)

palette = list(sns.color_palette("inferno_r", 6).as_hex())

# "Inferno-like" colormap with white background
white_inferno = LinearSegmentedColormap.from_list('white_inferno', [
    (0, '#ffffff'),
    (1e-10, palette[0]),
    (0.2, palette[1]),
    (0.4, palette[2]),
    (0.6, palette[3]),
    (0.8, palette[4]),
    (1, palette[5]),
], N=256)

# Define methods

## DOS interpolation

In [ ]:
@jit(nopython=True)
def fermi(E,mu,kbT):
    return 1.0 / ( np.exp( (E-mu) / kbT) + 1)

@jit(nopython=True)
def dfermi(E,mu,kbT):
    return 1.0/(2*kbT + 2*kbT*np.cosh((E - mu)/kbT))

@jit(nopython=True)
def dos(mu, kbT, spec):
       
    dens = 0.0
    
    for level in spec:
        dens += dfermi(level,mu,kbT)
        
    return dens / ( len(spec) / 2.0)

@jit(nopython=True)
def ids(mu, spec):
       
    dens = 0.0 
    
    for level in spec:
        if level < mu:
            dens += 1
        
    return dens / ( len(spec) / 2.0)

# Import data

## Read files for parameters

In [ ]:
files = [x[0] for x in os.walk('./data/')][1::]
df = pd.DataFrame()
for file in files:
    data = pd.read_json(file+'/params.json',orient='index').transpose()
    data["key"] = file[7:]
    
    if "n_shift" in data and "mag" in data and "tau1tau2" in data:
            df = pd.concat([df,data])
            
display(df.set_index('key').sort_index(ascending=False))

## Select data

### skx

In [ ]:
key = '1701375002'

## Import file and extract data

In [ ]:
outdir = './data/' + key

# parameter
tex, size = df.set_index('key').at[key,'texture'], df.set_index('key').at[key,'system_sizes'],
t, m = df.set_index('key').at[key,'t'], df.set_index('key').at[key,'m'],
theta, ids, mag = df.set_index('key').at[key,'q'], df.set_index('key').at[key,'ids'], df.set_index('key').at[key,'mag']
n_shift, min_shift, max_shift = df.set_index('key').at[key,'n_shift'], df.set_index('key').at[key,'min_shift'], df.set_index('key').at[key,'max_shift']
tau1tau2, u1u2, tau1tau2u1u2 = df.set_index('key').at[key,'tau1tau2'], df.set_index('key').at[key,'u1u2'], df.set_index('key').at[key,'tau1tau2u1u2']
phi, eta = df.set_index('key').at[key,'phi'], df.set_index('key').at[key,'eta']

# data
shifts = np.linspace(min_shift,max_shift,n_shift)
    
fermis = np.load(outdir+"/fermi.npy")
ch_tau1tau2 = np.load(outdir+"/ch_tau1tau2.npy")
ch_u1u2 = np.load(outdir+"/ch_u1u2.npy")
ch_tau1tau2u1u2 = np.load(outdir+"/ch_tau1tau2u1u2.npy")

# Full spectrum
spec   = np.array([np.load(outdir+'/spec_'+str(i).zfill(4)+'.npy') for i in range(n_shift)])

# Density array
x = []
y = []

for i,s in enumerate(spec):
    for e in s:
        x.append(shifts[i])
        y.append(e)

x = np.array(x)
y = np.array(y)

# DoS + Chern

In [ ]:
# Plot parameter
mpl.pyplot.rcdefaults()
plt.rcParams['figure.figsize'] = [12, 15]
plt.rcParams['savefig.facecolor'] = "white"
mpl.rcParams['figure.dpi'] = 300
mpl.rcParams['axes.linewidth'] = 1
mpl.rcParams['mathtext.fontset'] = 'stix'
mpl.rcParams['font.family'] = 'STIXGeneral'

tfs    = 48 #title font size
lfs    = 40 #label font size
fs     = 40 #font size
cbarfs = 18 #colorbar font size

# scatter size
s = 4

# figure
fig = plt.figure()
gs = fig.add_gridspec(5,1, hspace=0,wspace=0)
ax0 = plt.subplot(gs.new_subplotspec((0, 0), rowspan=2),projection='scatter_density')
# ax1 = plt.subplot(gs.new_subplotspec((2, 0), rowspan=1),sharex=ax0)
ax2 = plt.subplot(gs.new_subplotspec((2, 0), rowspan=1),sharex=ax0)
ax3 = plt.subplot(gs.new_subplotspec((3, 0), rowspan=1),sharex=ax0)
ax4 = plt.subplot(gs.new_subplotspec((4, 0), rowspan=1),sharex=ax0)

y_ticks = np.arange(-3, 2, 1)

ax0.scatter_density(x, y, cmap=white_blues,vmin=0,vmax=4.0, dpi=150)
ax0.set_yticks(y_ticks)
ax0.set_xlabel(r"$\varphi$",fontsize=lfs)
ax0.set_ylabel(r"$E_F$",fontsize=lfs)
ax0.plot(shifts,fermis, color = 'red', linestyle='dashed', label = r'fermi', linewidth=1)
ax0.set_ylim([-2.5, 1.5])

# y_ticks = np.arange(0.9, 1.1, 0.001)

# ax1.scatter(shifts,ch_0.real,s=s);
# ax1.set_yticks(y_ticks)
# ax1.grid(True)
# ax1.set_ylim([1.0291, 1.0319])
# ax1.set_xlabel(r'phase shift',fontsize=lfs)
# ax1.set_title(r'IDS',fontsize=tfs, x=0.007, y=0.88, loc='left',backgroundcolor= 'white')

y_ticks = np.arange(-1, 1.5, 0.5)

ax2.scatter(shifts,ch_tau1tau2.real,s=s);
ax2.set_yticks(y_ticks)
ax2.grid(True)
ax2.set_ylim([-1.2, 0.2])
ax2.set_xlabel(r'$\varphi$',fontsize=lfs)
ax2.set_title(r'$Ch_{t_1 t_2}$',fontsize=tfs, x=0.86, y=0.77, loc='left',backgroundcolor= 'white')

y_ticks = np.arange(-2, 1, 0.5)

ax3.scatter(shifts,ch_u1u2.real,s=s);
ax3.set_yticks(y_ticks)
ax3.grid(True)
ax3.set_ylim([-2.2, -0.8])
ax3.set_xlabel(r'$\varphi$',fontsize=lfs)
ax3.set_title(r'$Ch_{u_1 u_2}$',fontsize=tfs, x=0.8405, y=0.39, loc='left',backgroundcolor= 'white')

y_ticks = np.arange(-1.1, -0.9, 0.1)

ax4.scatter(shifts,ch_tau1tau2u1u2.real,s=s);
ax4.set_yticks(y_ticks)
ax4.grid(True)
ax4.set_ylim([-1.15, -0.85])
ax4.set_xlabel(r'$\varphi$',fontsize=lfs)
ax4.set_title(r'$Ch_{t_1 t_2 u_1 u_2}$',fontsize=tfs, x=0.78, y=0.77, loc='left',backgroundcolor= 'white')

for ax in fig.get_axes():
    ax.tick_params(axis='both', which='major', labelsize=fs)
    ax.set_xticks([n for n in np.linspace(0.1*np.pi,0.9*np.pi,5)])
    ax.xaxis.set_minor_locator(AutoMinorLocator())
    labels = [r'${:.1f} \pi$'.format(n) for n in np.linspace(0.1,0.9,5)]
    ax.set_xticklabels(labels)
    
for ax in fig.get_axes()[:-1]:
    ax.tick_params(bottom=False, labelbottom=False)
    
plt.tight_layout()
plt.savefig("./Plots/Chernhierarchy_phaseshift_07mag_reg01.png", dpi=300, bbox_inches = 'tight')

# Low resolution

In [ ]:
# Plot parameter
mpl.pyplot.rcdefaults()
plt.rcParams['figure.figsize'] = [12, 15]
plt.rcParams['savefig.facecolor'] = "white"
mpl.rcParams['figure.dpi'] = 100
mpl.rcParams['axes.linewidth'] = 1
mpl.rcParams['mathtext.fontset'] = 'stix'
mpl.rcParams['font.family'] = 'STIXGeneral'

tfs    = 48 #title font size
lfs    = 40 #label font size
fs     = 40 #font size
cbarfs = 18 #colorbar font size

# scatter size
s = 4

# figure
fig = plt.figure()
gs = fig.add_gridspec(5,1, hspace=0,wspace=0)
ax0 = plt.subplot(gs.new_subplotspec((0, 0), rowspan=2),projection='scatter_density')
# ax1 = plt.subplot(gs.new_subplotspec((2, 0), rowspan=1),sharex=ax0)
ax2 = plt.subplot(gs.new_subplotspec((2, 0), rowspan=1),sharex=ax0)
ax3 = plt.subplot(gs.new_subplotspec((3, 0), rowspan=1),sharex=ax0)
ax4 = plt.subplot(gs.new_subplotspec((4, 0), rowspan=1),sharex=ax0)

y_ticks = np.arange(-3, 2, 1)

ax0.scatter_density(x, y, cmap=white_blues,vmin=0,vmax=4.0, dpi=150)
ax0.set_yticks(y_ticks)
ax0.set_xlabel(r"$\varphi$",fontsize=lfs)
ax0.set_ylabel(r"$E_F$",fontsize=lfs)
ax0.plot(shifts,fermis, color = 'red', linestyle='dashed', label = r'fermi', linewidth=1)
ax0.set_ylim([-2.5, 1.5])

# y_ticks = np.arange(0.9, 1.1, 0.001)

# ax1.scatter(shifts,ch_0.real,s=s);
# ax1.set_yticks(y_ticks)
# ax1.grid(True)
# ax1.set_ylim([1.0291, 1.0319])
# ax1.set_xlabel(r'phase shift',fontsize=lfs)
# ax1.set_title(r'IDS',fontsize=tfs, x=0.007, y=0.88, loc='left',backgroundcolor= 'white')

y_ticks = np.arange(-1, 1.5, 0.5)

ax2.scatter(shifts,ch_tau1tau2.real,s=s);
ax2.set_yticks(y_ticks)
ax2.grid(True)
ax2.set_ylim([-1.2, 0.2])
ax2.set_xlabel(r'$\varphi$',fontsize=lfs)
ax2.set_title(r'$Ch_{t_1 t_2}$',fontsize=tfs, x=0.8585, y=0.766, loc='left',backgroundcolor= 'white')

y_ticks = np.arange(-2, 1, 0.5)

ax3.scatter(shifts,ch_u1u2.real,s=s);
ax3.set_yticks(y_ticks)
ax3.grid(True)
ax3.set_ylim([-2.2, -0.8])
ax3.set_xlabel(r'$\varphi$',fontsize=lfs)
ax3.set_title(r'$Ch_{u_1 u_2}$',fontsize=tfs, x=0.8385, y=0.386, loc='left',backgroundcolor= 'white')

y_ticks = np.arange(-1.1, -0.9, 0.1)

ax4.scatter(shifts,ch_tau1tau2u1u2.real,s=s);
ax4.set_yticks(y_ticks)
ax4.grid(True)
ax4.set_ylim([-1.15, -0.85])
ax4.set_xlabel(r'$\varphi$',fontsize=lfs)
ax4.set_title(r'$Ch_{t_1 t_2 u_1 u_2}$',fontsize=tfs, x=0.778, y=0.764, loc='left',backgroundcolor= 'white')

for ax in fig.get_axes():
    ax.tick_params(axis='both', which='major', labelsize=fs)
    ax.set_xticks([n for n in np.linspace(0.1*np.pi,0.9*np.pi,5)])
    ax.xaxis.set_minor_locator(AutoMinorLocator())
    labels = [r'${:.1f} \pi$'.format(n) for n in np.linspace(0.1,0.9,5)]
    ax.set_xticklabels(labels)
    
for ax in fig.get_axes()[:-1]:
    ax.tick_params(bottom=False, labelbottom=False)
    
plt.tight_layout()
plt.savefig("./Plots/lowres/Chernhierarchy_phaseshift_07mag_reg01.png", dpi=100, bbox_inches = 'tight')